In [1]:
!pip install -qq transformers
!pip install -qq optuna
!pip install -qq datasets

     |████████████████████████████████| 3.5 MB 14.4 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 6.8 MB 59.4 MB/s 
     |████████████████████████████████| 895 kB 61.3 MB/s 
     |████████████████████████████████| 596 kB 59.7 MB/s 
     |████████████████████████████████| 308 kB 15.3 MB/s 
     |████████████████████████████████| 210 kB 87.8 MB/s 
     |████████████████████████████████| 80 kB 11.3 MB/s 
     |████████████████████████████████| 75 kB 5.2 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 113 kB 67.9 MB/s 
     |████████████████████████████████| 149 kB 68.1 MB/s 
     |████████████████████████████████| 311 kB 13.6 MB/s 
     |████████████████████████████████| 243 kB 69.6 MB/s 
     |████████████████████████████████| 1.1 MB 79.3 MB/s 
     |████████████████████████████████| 133 kB 96.2 MB/s 
     |████████████████████████████████| 144 kB 82.8 MB/s 
     |███████████████

In [2]:
import transformers
import datasets
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, AutoModelForSequenceClassification,AdamW, get_linear_schedule_with_warmup,Trainer, TrainingArguments
from transformers import DataCollator, DataCollatorForLanguageModeling,default_data_collator
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
import random
from textwrap import wrap
from datetime import datetime
from datasets import load_from_disk
from datasets import load_dataset
from datasets import Dataset
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [3]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
MODEL_NAME = "distilbert-base-uncased"
# max sequence length for each document/sentence sample
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE= 6.58e-5
WEIGHT_DECAY = 0.289
WARMUP_STEPS = 464
RANDOM_SEED=22




QA_OUTPUT_PATH= "/content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad"

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [9]:
def set_seed(seed):
    """Set all seeds to make results reproducible (deterministic mode).
       When seed is None, disables deterministic mode.
    :param seed: an integer to your choosing
    """
    if seed is not None:
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)
        random.seed(seed)

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  confusion_matrix = classification_report(labels, preds, digits=4,output_dict=True)
  return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'hate_f1': confusion_matrix["0"]["f1-score"],
        'hate_recall': confusion_matrix["0"]["recall"],
        'hate_precision': confusion_matrix["0"]["precision"],
        'offensive_f1': confusion_matrix["1"]["f1-score"],
        'offensive_recall': confusion_matrix["1"]["recall"],
        'offensive_precision': confusion_matrix["1"]["precision"],
        'normal_f1': confusion_matrix["2"]["f1-score"],
        'normal_recall': confusion_matrix["2"]["recall"],
        'normal_precision': confusion_matrix["2"]["precision"],    
  }


def seq_model_init():
  temp_model =  AutoModelForSequenceClassification.from_pretrained(QA_OUTPUT_PATH,num_labels=3).to(device)
  return temp_model

def qa_model_init():
  temp_model =  AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME).to(device)
  return temp_model



def timestamp():
    dateTimeObj = datetime.now()
    timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")
    print(timestampStr)




In [5]:
set_seed(RANDOM_SEED)




In [6]:
dataset_dfs = load_from_disk('/content/drive/MyDrive/Dissertation/datasets/hatetwit_'+str(1))

In [7]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results',          # output directory
    num_train_epochs=EPOCHS,              # total number of training epochs
    save_strategy ="epoch" ,
    per_device_train_batch_size=BATCH_SIZE,  # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,   # batch size for evaluation
    weight_decay= WEIGHT_DECAY,               # strength of weight decay
    learning_rate= LEARNING_RATE, 
    logging_dir='./disbert_hate_task/hyper/logs',     # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    evaluation_strategy="epoch",
    #eval_steps = 500     # evaluate each `logging_steps`
)

In [11]:
hyper_trainer = Trainer(
    model_init=seq_model_init,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset_dfs['train'],         # training dataset
    eval_dataset=dataset_dfs['validation'],          # evaluation dataset
    compute_metrics=compute_metrics     # the callback that computes metrics of interest
)

loading configuration file /content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

loading weights file /content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best

In [12]:
def hp_space_optuna(trial) :
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 5),
        "seed": trial.suggest_int("seed", 1, 40),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 500),
        "weight_decay": trial.suggest_float("weight_decay", 0, 0.3),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [ 8, 16, 32, 64]),
    }

In [13]:
best_trial = hyper_trainer.hyperparameter_search(n_trials=40, direction="maximize", backend="optuna", hp_space=hp_space_optuna)

[I 2022-02-13 12:54:05,467] A new study created in memory with name: no-name-18c76278-872b-4904-bc44-8d11353d1bf1
Trial:
loading configuration file /content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version":

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.797000,0.700663,0.703799,0.636078,0.639799,0.632761,0.621901,0.605634,0.639066,0.813528,0.823769,0.803539,0.472803,0.468880,0.476793
2,0.679800,0.647603,0.732561,0.675370,0.677571,0.673289,0.659195,0.650905,0.667699,0.826686,0.832655,0.820803,0.540230,0.536307,0.544211
3,0.649900,0.637425,0.739429,0.676274,0.687565,0.668291,0.665986,0.655936,0.676349,0.831304,0.859311,0.805064,0.531532,0.489627,0.581281


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert, sentence.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-0/checkpoint-1165
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-0/checkpoint-1165/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-0/checkpoint-1165/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert, sentence.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Savin

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.580300,0.556196,0.778064,0.730080,0.731921,0.728721,0.714002,0.723340,0.704902,0.856249,0.861163,0.851391,0.619989,0.601660,0.639471
2,0.489800,0.513696,0.797167,0.748144,0.762716,0.738745,0.748122,0.751509,0.744766,0.867444,0.895224,0.841336,0.628866,0.569502,0.702046
3,0.399800,0.519182,0.804894,0.756247,0.765966,0.750074,0.762141,0.773642,0.750977,0.876742,0.896705,0.857649,0.629859,0.579876,0.689273
4,0.320400,0.557025,0.797811,0.758508,0.751173,0.767863,0.763395,0.809859,0.721973,0.867867,0.847464,0.889277,0.644261,0.646266,0.642268
5,0.257700,0.596510,0.801460,0.757741,0.755783,0.761129,0.769009,0.803823,0.737085,0.873255,0.868567,0.877994,0.630959,0.610996,0.652270


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert, sentence.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-1/checkpoint-2330
Configuration saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-1/checkpoint-2330/config.json
Model weights saved in /content/drive/MyDrive/Dissertation/disbert_hate_task/hyper/results/run-1/checkpoint-2330/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert, sentence.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
Savin

...

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert, sentence.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
[I 2022-02-13 15:37:01,643] Trial 37 pruned. 
Trial:
loading configuration file /content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.558300,0.560672,0.771625,0.721089,0.723260,0.727613,0.714286,0.794769,0.648604,0.853189,0.844502,0.862056,0.595793,0.543568,0.659119


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert, sentence.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
[I 2022-02-13 15:39:49,473] Trial 38 pruned. 
Trial:
loading configuration file /content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/drive/MyDrive/Dissertation/disbert_hate_ml/results/best_model_squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Hate F1,Hate Recall,Hate Precision,Offensive F1,Offensive Recall,Offensive Precision,Normal F1,Normal Recall,Normal Precision
1,0.695800,0.650311,0.735780,0.678781,0.683367,0.675231,0.648791,0.620724,0.679515,0.830126,0.838578,0.821843,0.557427,0.566390,0.548744


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_type_ids_bert, __index_level_0__, input_ids_bert, attention_mask_bert, sentence.
***** Running Evaluation *****
  Num examples = 4659
  Batch size = 16
[I 2022-02-13 15:41:36,670] Trial 39 pruned. 


In [14]:
best_trial

BestRun(run_id='10', objective=10.05241054638603, hyperparameters={'learning_rate': 8.584684132528283e-05, 'num_train_epochs': 4, 'seed': 32, 'warmup_steps': 50, 'weight_decay': 0.10919253165395515, 'per_device_train_batch_size': 64})